# Chapter 7: Multi-Agent Systems

Key Takeaways:
- **Multi-Agent Systems** involve multiple specialized agents collaborating to achieve a complex goal.
- **CrewAI** orchestrates these agents, managing their roles, tasks, and data flow.
- This pattern enables **specialization**, where each agent focuses on what they do best (e.g., researching vs. writing).

### Heuristic: *Divide and conquer.*

## Setup and Initialization

In [ ]:
import os
import getpass
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

In [ ]:
# --- Configuration ---
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

try:
    # Initializes and runs the AI crew for content creation using the latest Gemini model.
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    print(f"✅ Language model initialized: {llm.model}")
except Exception as e:
    print(f"🛑 Error initializing language model: {e}")
    llm = None

## Define Agents

We define two specialized agents: a **Senior Research Analyst** and a **Technical Content Writer**.

In [ ]:
# --- Define Agents ---
if llm:
    # Researcher Agent
    researcher = Agent(
        role='Senior Research Analyst',
        goal='Find and summarize the latest trends in AI.',
        backstory="You are an experienced research analyst with a knack for identifying key trends and synthesizing information.",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )

    # Writer Agent
    writer = Agent(
        role='Technical Content Writer',
        goal='Write a clear and engaging blog post based on research findings.',
        backstory="You are a skilled writer who can translate complex technical topics into accessible content.",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )

## Define Tasks

The researcher finds the trends, and the writer creates the content based on those findings.

In [ ]:
# --- Define Tasks ---

# Research Task
research_task = Task(
    description="Research the top 3 emerging trends in Artificial Intelligence in 2024-2025. Focus on practical applications and potential impact.",
    expected_output="A detailed summary of the top 3 AI trends, including key points and sources.",
    agent=researcher,
)

# Writing Task
writing_task = Task(
    description="Write a 500-word blog post based on the research findings. The post should be engaging and easy for a general audience to understand.",
    expected_output="A complete 500-word blog post about the latest AI trends.",
    agent=writer,
    context=[research_task], # This explicitly passes the research output to the writer
)

## Execution

We group the agents into a `Crew` and execute the process sequentially.

In [ ]:
# --- Execute Crew ---
if llm:
    # Create the Crew
    blog_creation_crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task],
        process=Process.sequential,
        verbose=True # Set verbosity for detailed crew execution logs
    )

    print("## Running the blog creation crew with Gemini 2.0 Flash... ##")
    try:
        result = blog_creation_crew.kickoff()
        print("\n\n---\n## Crew Final Output ##\n---")
        print(result)
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

## Conclusion

By decomposing a workflow into specialized roles (Research -> Writing), we can achieve higher quality outputs than a single prompt might produce. The multi-agent approach mimics a real-world editorial team.